In [1]:
#!pip install pyspark

     |████████████████████████████████| 281.3 MB 41 kB/s 
     |████████████████████████████████| 198 kB 45.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=1bdfbc0415be81438a28300abc84de20f5f7dd4d61bdd7d8f8e3e327ead4454e
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [2]:
import pyspark

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
data = spark.read.csv('/content/drive/data/boston.csv', header = True, inferSchema = True)
data.printSchema()

root
 |-- CRIM: double (nullable = true)
 |-- ZN: double (nullable = true)
 |-- INDUS: double (nullable = true)
 |-- CHAS: integer (nullable = true)
 |-- NOX: double (nullable = true)
 |-- RM: double (nullable = true)
 |-- AGE: double (nullable = true)
 |-- DIS: double (nullable = true)
 |-- RAD: integer (nullable = true)
 |-- TAX: integer (nullable = true)
 |-- PT: double (nullable = true)
 |-- B: double (nullable = true)
 |-- LSTAT: double (nullable = true)
 |-- MV: double (nullable = true)



In [6]:
import pandas
pandas.DataFrame(data.take(5), columns=data.columns).transpose()

,0,1,2,3,4
CRIM,0.006320,0.027310,0.027290,0.032370,0.069050
ZN,18.000000,0.000000,0.000000,0.000000,0.000000
INDUS,2.310000,7.070000,7.070000,2.180000,2.180000
CHAS,0.000000,0.000000,0.000000,0.000000,0.000000
NOX,0.538000,0.469000,0.469000,0.458000,0.458000
RM,6.575000,6.421000,7.185000,6.998000,7.147000
AGE,65.199997,78.900002,61.099998,45.799999,54.200001
DIS,4.090000,4.967100,4.967100,6.062200,6.062200
RAD,1.000000,2.000000,2.000000,3.000000,3.000000
TAX,296.000000,242.000000,242.000000,222.000000,222.000000


In [16]:
data.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
CRIM,506,3.6135235608162057,8.601545086715594,0.00632,88.97619629
ZN,506,11.363636363636363,23.32245299451514,0.0,100.0
INDUS,506,11.136778749531626,6.86035298095724,0.460000008,27.73999977
CHAS,506,0.0691699604743083,0.2539940413404101,0,1
NOX,506,0.5546950602312246,0.1158776754570543,0.38499999,0.870999992
RM,506,6.28463438896641,0.7026171549511354,3.561000109,8.779999733
AGE,506,68.57490120115612,28.148861532793276,2.900000095,100.0
DIS,506,3.7950426960059325,2.105710142043288,1.129600048,12.12650013
RAD,506,9.549407114624506,8.707259384239366,1,24


In [18]:
import pandas as pd
numeric_features = [t[0] for t in data.dtypes if t[1] == 'int' or t[1] == 'double']

In [19]:
numeric_features

['CRIM',
 'ZN',
 'INDUS',
 'CHAS',
 'NOX',
 'RM',
 'AGE',
 'DIS',
 'RAD',
 'TAX',
 'PT',
 'B',
 'LSTAT',
 'MV']

In [20]:
sampled_data = data.select(numeric_features).sample(False, 0.8).toPandas()

In [21]:
sampled_data

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PT,B,LSTAT,MV
0,0.00632,18.0,2.31,0,0.538,6.575,65.199997,4.0900,1,296,15.300000,396.899994,4.98,24.000000
1,0.02731,0.0,7.07,0,0.469,6.421,78.900002,4.9671,2,242,17.799999,396.899994,9.14,21.600000
2,0.02729,0.0,7.07,0,0.469,7.185,61.099998,4.9671,2,242,17.799999,392.829987,4.03,34.700001
3,0.03237,0.0,2.18,0,0.458,6.998,45.799999,6.0622,3,222,18.700001,394.630005,2.94,33.400002
4,0.06905,0.0,2.18,0,0.458,7.147,54.200001,6.0622,3,222,18.700001,396.899994,5.33,36.200001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,0.17783,0.0,9.69,0,0.585,5.569,73.500000,2.3999,6,391,19.200001,395.769989,15.10,17.500000
384,0.22438,0.0,9.69,0,0.585,6.027,79.699997,2.4982,6,391,19.200001,396.899994,14.33,16.799999
385,0.04527,0.0,11.93,0,0.573,6.120,76.699997,2.2875,1,273,21.000000,396.899994,9.08,20.600000
386,0.10959,0.0,11.93,0,0.573,6.794,89.300003,2.3889,1,273,21.000000,393.450012,6.48,22.000000


In [26]:
import six


In [28]:
for i in data.columns:
    if not( isinstance(data.select(i).take(1)[0][0], six.string_types)):
        print( "Correlation to MV for ", i, data.stat.corr('MV',i))

Correlation to MV for  CRIM -0.3883046116575088
Correlation to MV for  ZN 0.36044534463752903
Correlation to MV for  INDUS -0.48372517128143383
Correlation to MV for  CHAS 0.17526017775291847
Correlation to MV for  NOX -0.4273207763683772
Correlation to MV for  RM 0.695359937127267
Correlation to MV for  AGE -0.37695456714288667
Correlation to MV for  DIS 0.24992873873512172
Correlation to MV for  RAD -0.3816262315669168
Correlation to MV for  TAX -0.46853593528654536
Correlation to MV for  PT -0.5077867038116085
Correlation to MV for  B 0.3334608226834164
Correlation to MV for  LSTAT -0.7376627294671615
Correlation to MV for  MV 1.0


In [9]:
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler


vector_col = "corr_features"
assembler = VectorAssembler(inputCols=data.columns, outputCol=vector_col)
data_vector = assembler.transform(data).select(vector_col)


matrix = Correlation.corr(data_vector, vector_col)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [10]:
print(matrix)

DataFrame[pearson(corr_features): matrix]


In [11]:
matrix.collect()[0]["pearson({})".format(vector_col)].values

array([ 1.        , -0.20046922,  0.40658343, -0.05589158,  0.42097173,
       -0.2192467 ,  0.35273425, -0.37967009,  0.62550515,  0.58276431,
        0.28994564, -0.38506395,  0.45562148, -0.38830461, -0.20046922,
        1.        , -0.53382819, -0.04269672, -0.51660371,  0.31199059,
       -0.56953734,  0.66440822, -0.31194783, -0.31456332, -0.39167853,
        0.17552031, -0.41299458,  0.36044534,  0.40658343, -0.53382819,
        1.        ,  0.06293803,  0.76365146, -0.39167586,  0.64477851,
       -0.70802699,  0.5951293 ,  0.7207602 ,  0.38324764, -0.35697654,
        0.60379972, -0.48372517, -0.05589158, -0.04269672,  0.06293803,
        1.        ,  0.0912028 ,  0.09125123,  0.08651777, -0.09917578,
       -0.00736824, -0.03558652, -0.12151517,  0.04878849, -0.0539293 ,
        0.17526018,  0.42097173, -0.51660371,  0.76365146,  0.0912028 ,
        1.        , -0.30218816,  0.73147011, -0.76923012,  0.61144056,
        0.66802321,  0.18893272, -0.38005064,  0.59087892, -0.42

In [29]:
vectorAssembler = VectorAssembler(inputCols = 
                                  ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PT', 'B', 'LSTAT'], outputCol = 'features')

In [31]:
vector_data  = vectorAssembler.transform(data)

In [32]:
vector_data

DataFrame[CRIM: double, ZN: double, INDUS: double, CHAS: int, NOX: double, RM: double, AGE: double, DIS: double, RAD: int, TAX: int, PT: double, B: double, LSTAT: double, MV: double, features: vector]

In [33]:
vector_data = vector_data.select(['features', 'MV'])

In [35]:
vector_data.show(5)

+--------------------+-----------+
|            features|         MV|
+--------------------+-----------+
|[0.00632,18.0,2.3...|       24.0|
|[0.027310001,0.0,...|21.60000038|
|[0.02729,0.0,7.07...|34.70000076|
|[0.032370001,0.0,...|33.40000153|
|[0.069049999,0.0,...|36.20000076|
+--------------------+-----------+
only showing top 5 rows



In [36]:
splits = vector_data.randomSplit([0.7, 0.3])
train_data = splits[0]
test_data = splits[1]

In [39]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='MV', maxIter=10, regParam=0.2, elasticNetParam=0.7)
lr_model = lr.fit(train_data)

In [42]:
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 5.090220
r2: 0.695929


In [43]:
train_data.describe().show()

+-------+------------------+
|summary|                MV|
+-------+------------------+
|  count|               363|
|   mean|22.377961458269976|
| stddev| 9.243747168806626|
|    min|               5.0|
|    max|              50.0|
+-------+------------------+



In [45]:
lr_predictions = lr_model.transform(test_data)
lr_predictions.select("prediction","MV","features").show(5)

+------------------+-----------+--------------------+
|        prediction|         MV|            features|
+------------------+-----------+--------------------+
|30.461453612686732|       24.0|[0.00632,18.0,2.3...|
|30.284820300434156|32.20000076|[0.00906,90.0,2.9...|
| 37.73463089889245|       50.0|[0.01381,80.0,0.4...|
|27.026001113195377|       24.5|[0.01501,80.0,2.0...|
| 41.19426506893861|       50.0|[0.01501,90.0,1.2...|
+------------------+-----------+--------------------+
only showing top 5 rows



In [46]:
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="MV",metricName="r2")

In [47]:
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

R Squared (R2) on test data = 0.769889


In [48]:
test_result = lr_model.evaluate(test_data)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)

Root Mean Squared Error (RMSE) on test data = 4.34899
